In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pandas_gbq

import requests
from bs4 import BeautifulSoup
import json
import lxml

from datetime import datetime, timedelta
import time

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, HeatMap

from shapely.geometry import Point, Polygon, MultiPolygon, shape
from shapely import wkt

import googlemaps
from google.cloud import bigquery
from google.oauth2 import service_account

import os
import glob
import warnings

DATA_PATH = "data/"
KEY_PATH = "config/"

key_path = KEY_PATH + "fireforest-team-ys-2023.json"
servicekey_path = KEY_PATH + "serviceKey.json"

warnings.filterwarnings("ignore")

In [2]:
def get_service_key(servicekey_path, key_name):
    """
    주어진 서비스 키 파일에서 지정된 키 이름에 해당하는 서비스 키를 반환합니다.

    Args:
        servicekey_path (str): 서비스 키 파일의 경로.
        key_name (str): 반환할 서비스 키의 이름.

    Returns:
        str or None: 지정된 키 이름에 해당하는 서비스 키. 키를 찾을 수 없는 경우 None을 반환합니다.
    """
    
    with open(servicekey_path) as f:
        data = json.load(f)
        service_key = data.get(key_name)
    return service_key

In [3]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    """
    주어진 데이터프레임을 BigQuery 테이블에 저장합니다.

    Args:
        df (pandas.DataFrame): 저장할 데이터프레임.
        dataset_id (str): 대상 데이터셋의 ID.
        table_id (str): 대상 테이블의 ID.
        key_path (str): 서비스 계정 키 파일의 경로.

    Returns:
        None
    """
    
    # Credentials 객체 생성
    credentials = service_account.Credentials.from_service_account_file(key_path)

    # 빅쿼리 클라이언트 객체 생성
    client = bigquery.Client(credentials=credentials)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [4]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    """
    주어진 BigQuery 테이블에서 데이터를 조회하여 DataFrame으로 반환합니다.

    Args:
        dataset_id (str): 대상 데이터셋의 ID.
        table_id (str): 대상 테이블의 ID.
        key_path (str): 서비스 계정 키 파일의 경로.

    Returns:
        pandas.DataFrame: 조회된 데이터를 담은 DataFrame 객체.
    """

    # Credentials 객체 생성
    credentials = service_account.Credentials.from_service_account_file(key_path)

    # BigQuery 클라이언트 생성
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [5]:
def save_geodataframe_to_bigquery(gdf, dataset_id, table_id, key_path):
    """
    주어진 Geopandas GeoDataFrame을 BigQuery 테이블에 저장합니다.

    Args:
        gdf (geopandas.GeoDataFrame): 저장할 Geopandas GeoDataFrame 객체.
        dataset_id (str): 대상 데이터셋의 ID.
        table_id (str): 대상 테이블의 ID.
        key_path (str): 서비스 계정 키 파일의 경로.

    Returns:
        None
    """
        
    gdf = gdf.to_crs('EPSG:4326')
    gdf['geometry'] = gdf['geometry'].astype(str)

    # Geopandas GeoDataFrame을 Pandas DataFrame으로 변환
    df = pd.DataFrame(gdf)

    # Credentials 객체 생성
    credentials = service_account.Credentials.from_service_account_file(key_path)

    # 빅쿼리 클라이언트 객체 생성
    client = bigquery.Client(credentials=credentials)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [6]:
def get_geodataframe_from_bigquery(dataset_id, table_id, key_path):
    """
    주어진 BigQuery 테이블에서 데이터를 조회하여 Geopandas GeoDataFrame으로 반환합니다.

    Args:
        dataset_id (str): 대상 데이터셋의 ID.
        table_id (str): 대상 테이블의 ID.
        key_path (str): 서비스 계정 키 파일의 경로.

    Returns:
        geopandas.GeoDataFrame: 조회된 데이터를 담은 Geopandas GeoDataFrame 객체.
    """
    
    # Credentials 객체 생성
    credentials = service_account.Credentials.from_service_account_file(key_path)

    # 빅쿼리 클라이언트 객체 생성
    client = bigquery.Client(credentials=credentials)

    # 쿼리 작성
    query = f"SELECT * FROM `{dataset_id}.{table_id}`"

    # 쿼리 실행
    df = client.query(query).to_dataframe()
    
    # 'geometry' 열의 문자열을 다각형 객체로 변환
    df['geometry'] = df['geometry'].apply(wkt.loads)

    # GeoDataFrame으로 변환
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = "EPSG:4326"

    return gdf

In [7]:
def get_weather_days_data(serviceKey, weather_stations, start_date_str=None, end_date_str=None):
    """
    지정한 기상 관측소의 일별 날씨 데이터를 조회하여 데이터프레임으로 반환합니다.

    Args:
        serviceKey (str): 공공데이터포털에서 발급받은 인증키.
        weather_stations (pandas.DataFrame): 기상 관측소 정보가 포함된 데이터프레임.
        start_date_str (str, optional): 조회 시작 날짜를 나타내는 문자열 (예: "20220101").
            기본값은 None이며, 기본값일 경우 2013년 1월 1일로 설정됩니다.
        end_date_str (str, optional): 조회 끝 날짜를 나타내는 문자열 (예: "20220331").
            기본값은 None이며, 기본값일 경우 어제 날짜로 설정됩니다.

    Returns:
        pandas.DataFrame: 조회된 일별 날씨 데이터를 담은 데이터프레임 객체.
    """
        
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
    
    # 시작 날짜와 끝 날짜를 생성합니다
    if start_date_str is None:
        start_date = datetime(2013, 1, 1)  # 시작 날짜를 2013년 1월 1일로 설정합니다
    else:
        start_date = datetime.strptime(start_date_str, "%Y%m%d")
    
    if end_date_str is None:
        end_date = datetime.now() - timedelta(days=1)  # 어제 날짜를 구하기 위해 현재 날짜에서 1일을 뺍니다
    else:
        end_date = datetime.strptime(end_date_str, "%Y%m%d")
    
    end_date_str = end_date.strftime("%Y%m%d")

    all_data = []  # 전체 데이터를 저장할 리스트를 생성합니다

    for stnNm in weather_stations["stnNm"]:
        params ={
            'serviceKey' : serviceKey, 
            'pageNo' : '1',  # 초기 페이지 번호를 1로 설정합니다
            'numOfRows' : '999',  # 한 페이지에 최대로 가져올 데이터 수를 설정합니다
            'dataType' : 'json',
            'dataCd' : 'ASOS',
            'dateCd' : 'DAY',
            'startDt' : start_date.strftime("%Y%m%d"),  # 시작 날짜를 문자열로 변환하여 설정합니다
            'endDt' : end_date_str,  # 끝 날짜를 어제 날짜로 설정합니다
            'stnIds' : stnNm 
        }

        while True:
            try:
                response = requests.get(url, params=params)
                response.raise_for_status()  # 오류가 발생하면 예외를 발생시킴
                data = response.json()
                all_data.extend(data['response']['body']['items']['item'])

                # 다음 페이지로 이동
                params['pageNo'] = str(int(params['pageNo']) + 1)
                if int(params['pageNo']) > int(int(data['response']['body']['totalCount']) / int(params['numOfRows'])) + 1:
                    break
            except requests.exceptions.HTTPError as e:
                print("API 요청 오류:", e.response.text)  # API 요청 오류 메시지 출력
                break
            except Exception as e:
                print(params)
                print(response.content)
                print("예외 발생:", e)  # 기타 예외 발생 시 메시지 출력
                break

    # 리스트에서 데이터프레임을 생성합니다
    weather_days = pd.DataFrame(all_data)
    
    return weather_days

In [8]:
def get_forestfire_occurs_data(serviceKey):
    """
    산불 발생 데이터를 조회하여 데이터프레임으로 반환합니다.

    Args:
        serviceKey (str): safemap 에서 발급받은 인증키.

    Returns:
        pandas.DataFrame: 조회된 산불 발생 데이터를 담은 데이터프레임 객체.
    """
    
    url = "http://safemap.go.kr/openApiService/data/getFrfireSttusData.do"
    params = {
        "serviceKey": serviceKey,
        "pageNo": "1",
        "numOfRows": "9999",
        "type": "xml"
    }

    response = requests.get(url, params)
    soup = BeautifulSoup(response.content, "lxml")  # XML 파싱

    columns = [
        'objt_id', 'occu_year', 'occu_mt', 'occu_de', 'occu_tm', 'occu_day',
        'occu_date', 'end_year', 'end_mt', 'end_de', 'end_tm', 'adres',
        'rn_adres', 'resn', 'ar', 'amount', 'ctprvn_cd', 'sgg_cd', 'emd_cd',
        'x', 'y'
    ]
    
    data = {}
    for column in columns:
        data[column] = [element.get_text() for element in soup.find_all(column)]
    
    forestfire_occurs = pd.DataFrame(data)
    
    return forestfire_occurs

In [9]:
def get_forestfire_occurs_add_data(serviceKey):
    """
    산불 추가 발생 데이터를 조회하여 데이터프레임으로 반환합니다.

    Args:
        serviceKey (str): 공공데이터포털에서 발급받은 인증키.

    Returns:
        pandas.DataFrame: 조회된 산불 추가 발생 데이터를 담은 데이터프레임 객체.
    """
        
    url = 'http://apis.data.go.kr/1400000/forestStusService/getfirestatsservice'
    
    end_date = datetime.now() - timedelta(days=1)
    end_date_str = end_date.strftime("%Y%m%d")

    params = {
        'serviceKey' : serviceKey,
        'numOfRows' : '9999',
        'pageNo' : '1',
        'searchStDt' : '20220101',
        'searchEdDt' : end_date_str
    }

    response = requests.get(url, params)
    soup = BeautifulSoup(response.content, "lxml") #XML 파싱

    columns = [
        'damagearea', 'endday', 'endmonth', 'endyear', 'firecause', 'locbunji',
        'locdong', 'locgungu', 'locmenu', 'locsi', 'startday', 'startdayofweek',
        'startmonth', 'starttime', 'startyear'
    ]
    
    data = {}
    for column in columns:
        data[column] = [element.get_text() for element in soup.find_all(column)]
    
    forestfire_occurs = pd.DataFrame(data)
    
    return forestfire_occurs

In [10]:
# dataload

weather_stations = pd.read_csv(DATA_PATH + "weather_stations.csv", encoding="cp949")
weather_days = pd.read_csv(DATA_PATH + "weather_days.csv", encoding="cp949")

forestfire_occurs = pd.read_csv(DATA_PATH + "forestfire_occurs.csv", encoding="cp949", dtype="object")
forestfire_occurs_add = pd.read_csv(DATA_PATH + "forestfire_occurs_concat.csv", encoding="cp949", dtype="object")

gangwon_SGG = gpd.read_file(DATA_PATH + "시군구_강원/LARD_ADM_SECT_SGG_42.shp", encoding='cp949')
gangwon_UMD = gpd.read_file(DATA_PATH + "읍면동(법정동)_강원/LSMD_ADM_SECT_UMD_42.shp", encoding='cp949')
gangwon_code = pd.read_csv(DATA_PATH + "gangwon_code.csv", encoding="cp949")